In [1]:
%run "../Common/Enums"

In [2]:
def CopyDynamicsData(lakeName):
  print("LakeName:", lakeName)
  
  # get last date from the loaded dataset
  LastSyncDate = GetLastSyncDate('audits', 'dynamics')
  
  print("Reading data since", LastSyncDate)  
  print("Load Audits data")
  PrepareDataBronzeWithoutInstance (
    lakeName = lakeName,
    entity = BronzeTable.Audits,
    entityName = 'tmpAudits',
    startDate = LastSyncDate
  )
  
  print("Writing data...")
  
  sourceData = spark.sql("""
    select
          a.ObjectId
        , a.UserId
        , a.Operation
        , coalesce(au.OperationName, 'BAD DATA') as OperationName
        , a.CreatedOn
        , a.AuditId
        , a.Action
        , coalesce(aa.ActionName, 'BAD DATA') as ActionName
        , a.ObjectTypeCode
        , current_timestamp() as SilverModifiedDate
    from tmpAudits a
      left join tmpAuditOperation au on au.OperationId = a.Operation
      left join tmpAuditAction aa on aa.ActionId = a.Action
  """)  
  
  sourceData.registerTempTable("SourceData")
   
  spark.sql("""
    merge into delta.`wasbs://silver@"""+lakeName+""".blob.core.windows.net/dynamics/audits` as t
    using SourceData as s
      on t.AuditId = s.AuditId
    when matched then update set 
        ObjectId = s.ObjectId
      , UserId = s.UserId
      , Operation = s.Operation
      , OperationName = s.OperationName
      , CreatedOn = s.CreatedOn
      , Action = s.Action
      , ActionName = s.ActionName
      , ObjectTypeCode = s.ObjectTypeCode
      , SilverModifiedDate = s.SilverModifiedDate
    when not matched then 
      insert ( AuditId, ObjectId, UserId, Operation, OperationName, CreatedOn, Action, ActionName, ObjectTypeCode, SilverModifiedDate )
      values ( AuditId, ObjectId, UserId, Operation, OperationName, CreatedOn, Action, ActionName, ObjectTypeCode, SilverModifiedDate )
  """);

  print("Writing data completed.")
  
  LastSyncDate = sourceData.agg({"CreatedOn": "max"}).collect()[0][0]
  print(LastSyncDate)

  if LastSyncDate is not None:
    UpdateLastSyncDate(LastSyncDate, 'audits', 'dynamics')